# Assignment - Basic Pandas
<sup>Created by Natawut Nupairoj, Department of Computer Engineering, Chulalongkorn University</sup>

Using pandas to explore youtube trending data from GB (GBvideos.csv and GB_category_id.json) and answer the questions.

In [1]:
import pandas as pd
import numpy as np

To simplify data retrieval process on Colab, we heck if we are in the Colab environment and download data files from a shared drive and save them in folder "data".

For those using jupyter notebook on the local computer, you can read data directly assuming you save data in the folder "data".

In [2]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !wget https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/datasets/data.tgz -O data.tgz
    !tar -xzvf data.tgz

--2024-01-15 06:17:47--  https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/datasets/data.tgz
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kaopanboonyuen/2110446_DataScience_2021s2/main/datasets/data.tgz [following]
--2024-01-15 06:17:48--  https://raw.githubusercontent.com/kaopanboonyuen/2110446_DataScience_2021s2/main/datasets/data.tgz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45477462 (43M) [application/octet-stream]
Saving to: ‘data.tgz’

data.tgz            100%[===================>]  43.37M   158MB/s    in 0.3s    

2024-01-15 06:17:50 (158 MB/s) - ‘data.tgz’ sa

## How many rows are there in the GBvideos.csv after removing duplications?

In [9]:
gbv_df = pd.read_csv('data/GBvideos.csv')
gbv_df.drop_duplicates(keep='first', inplace=True)
print(len(gbv_df))

38745


## How many VDO's that contain at least one record (row) of "dislikes" more than "likes"?  <font color=red>DO NOT group by the data and make sure that you count only unique title!</font>

In [11]:
print(gbv_df[gbv_df.dislikes > gbv_df.likes].title.nunique())

56


## How many VDO that are trending on 22 Jan 2018 with comments more than 10,000 comments?

In [12]:
print(len(gbv_df[(gbv_df.trending_date == '18.22.01') & (gbv_df.comment_count > 10000)]))

28


## Which date that has the minimum average number of comments per VDO?

In [13]:
groupbydate_df = gbv_df.groupby('trending_date').comment_count.mean()
min_comments = groupbydate_df.min()
print(groupbydate_df[groupbydate_df == min_comments])

trending_date
17.15.11    4738.673367
Name: comment_count, dtype: float64


## Compare "Sports" and "Comady", how many days that there are more total daily views of VDO in "Sports" category than in "Comady" category?

In [18]:
import json
with open('data/US_category_id.json') as fd:
  categories = json.load(fd)

sports = []
comedy = []
for cat in categories['items']:
  if cat['snippet']['title'] == 'Sports':
    sports.append(int(cat['id']))
  elif cat['snippet']['title'] == 'Comedy':
    comedy.append(int(cat['id']))

dailyview_df = gbv_df.groupby(['trending_date', 'category_id'])['views'].sum()
sport_daily = dailyview_df[dailyview_df.index.get_level_values('category_id').isin(sports)]
comedy_daily = dailyview_df[dailyview_df.index.get_level_values('category_id').isin(comedy)]
print(len(sport_daily.groupby('trending_date').sum()[sport_daily.groupby('trending_date').sum() > comedy_daily.groupby('trending_date').sum()]))

49
